In [15]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import models
from torchvision import transforms
from collections import OrderedDict

In [3]:
# Pretrained model
vgg19 = models.vgg19(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /home/joachimiak/.torch/models/vgg19-dcbb9e9d.pth
100%|██████████| 574673361/574673361 [00:17<00:00, 31967925.28it/s]


In [4]:
VG = '../dumps/vg.jpg'
TUE = '../dumps/tue.jpg'

In [5]:
# Loading image
import matplotlib.pyplot as plt
from torchvision import transforms

def process(path):
    img = plt.imread(path)
    transformer = transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225])
        ]
    )
    return transformer(img)

def show_image(img):
    img = img.numpy()
    img = img.swapaxes(0, 2)
    img = img.swapaxes(0, 1)
    print img.shape
    plt.imshow(img)
    plt.show()

In [11]:
tue = process(TUE)
vg = process(VG)
#show_image(img)

tue = tue.unsqueeze(0)
vg = vg.unsqueeze(0)

In [12]:
# Training
style = torch.autograd.Variable(vg) # Style from Van-Gogh image
content = torch.autograd.Variable(tue) # Content from TUE
output = torch.rand(vg.size())

In [14]:
# Function to be recomputed in each step

def features(data):
    features = []
    
    def save()



torch.Size([1, 3, 256, 256])